# DEPENDÊNCIAS

In [ ]:
#! pip install spacy
#! pip install nltk
#! pip install pandas
#! pip install pdfplumber
#! pip install chromadb
#! pip install sentence-transformers
#! pip install openpyxl
#! pip install python-docx
#! pip install langchain-text-splitters
#! python -m spacy download pt_core_news_sm

# IMPORTS

In [91]:
import pdfplumber
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import spacy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import chromadb

In [92]:
pdf_path = "aprendizado de maquina.pdf"

# Carrega PDF

In [93]:
# utiliza a library pdfplumber para extrair o texto do PDF, 
# o resultado é guardado na variável raw_text
def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        lines = 0;
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
            lines += text.count("\n")

        print("Linhas:", lines)
        print("Caractéres:", len(pdf.chars))
    return text



raw_text = load_pdf(pdf_path)
print("Prévia do PDF (Primeiros 500 caractéres):\n\n" + raw_text[:500])



Linhas: 26
Caractéres: 1903
Prévia do PDF (Primeiros 500 caractéres):

APRENDIZAGEM DE MÁQUINA
O aprendizado de máquina (AM) é um segmento da Inteligência Artificial que possui
um elemento essencial para um comportamento inteligente, a capacidade de
aprendizado. A área de AM é responsável por pesquisar métodos computacionais
adequados para a aquisição de novos conhecimentos, novas habilidades e novas
formas de organização do conhecimento já existente. O aprendizado possibilita que
o sistema faça a mesma tarefa ou tarefas sobre uma mesma população de uma
maneira mai


# PLN

## Lowercasing

In [94]:
# transforma todo o texto em letras minúsculas
# isso é feito para normalizar o texto, e criar menos variação.
text_lower = raw_text.lower()




print("Texto em lowercase (prévia):\n\n", text_lower[:800])

Texto em lowercase (prévia):

 aprendizagem de máquina
o aprendizado de máquina (am) é um segmento da inteligência artificial que possui
um elemento essencial para um comportamento inteligente, a capacidade de
aprendizado. a área de am é responsável por pesquisar métodos computacionais
adequados para a aquisição de novos conhecimentos, novas habilidades e novas
formas de organização do conhecimento já existente. o aprendizado possibilita que
o sistema faça a mesma tarefa ou tarefas sobre uma mesma população de uma
maneira mais eficiente a cada execução.
o campo do aprendizado de máquina é concebido pela questão de como construir
programas, que automaticamente melhoram com a sua experiência [michell,
1997]. segundo batista: “aprendizado de máquina – am – é uma subárea de
pesquisa muito importante em inteligência artifici


## Remover caracteres especiais

In [95]:
def remove_special_chars(text):
    # normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # remove caracteres que não sejam letras, numeros ou espaços
    # deixo também ponto-final e virgula, para usar o 
    # método de chunk recursivo posteriormente
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()




text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais:\n\n", text_clean[:800])


Texto sem caracteres especiais:

 aprendizagem de maquina o aprendizado de maquina am e um segmento da inteligencia artificial que possui um elemento essencial para um comportamento inteligente, a capacidade de aprendizado. a area de am e responsavel por pesquisar metodos computacionais adequados para a aquisicao de novos conhecimentos, novas habilidades e novas formas de organizacao do conhecimento ja existente. o aprendizado possibilita que o sistema faca a mesma tarefa ou tarefas sobre uma mesma populacao de uma maneira mais eficiente a cada execucao. o campo do aprendizado de maquina e concebido pela questao de como construir programas, que automaticamente melhoram com a sua experiencia michell, 1997 . segundo batista aprendizado de maquina am e uma subarea de pesquisa muito importante em inteligencia artificial ia poi


## Remover Stopwords

In [96]:
# tokenização com spaCy pois funciona melhor com português, em comparação com nltk.
# stopwords foram resgatadas com o NLTK, 
# pois a lista de stopwords em português do spaCy 
# não foi tão boa quanto a do NLTK nos testes que fiz



# carrega o modelo de português
nlp = spacy.load("pt_core_news_sm")

# tokeniza utilizando spacy
token = nlp(text_clean)
tokens_with_stop = [t for t in token]

# baixa os recursos necessários do NLTK para stopwords, e as remove.
# PS: stopwords são palavras que não agregam para o contexto, e podem ser filtradas.
# palavras como ["a", "o", "de", "da", "na", etc...]
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))

# é possível adicionar stopwords ao conjunto, por exemplo, nesse texto existe uma referência: (2003, p. 11)
# caso queira filtrar o "p.", que não agrega muito valor, basta fazer isto:
stopwords_pt.add("p.")

tokens_no_stop = [t for t in token if t.text not in stopwords_pt]

print("\nTokens (todos) (primeiros 30):       ", tokens_with_stop[:30])
print("Tokens sem stopwords (primeiros 30): ", tokens_no_stop[:30])
print("\n")



Tokens (todos) (primeiros 30):        [aprendizagem, de, maquina, o, aprendizado, de, maquina, am, e, um, segmento, da, inteligencia, artificial, que, possui, um, elemento, essencial, para, um, comportamento, inteligente, ,, a, capacidade, de, aprendizado, ., a]
Tokens sem stopwords (primeiros 30):  [aprendizagem, maquina, aprendizado, maquina, am, segmento, inteligencia, artificial, possui, elemento, essencial, comportamento, inteligente, ,, capacidade, aprendizado, ., area, am, responsavel, pesquisar, metodos, computacionais, adequados, aquisicao, novos, conhecimentos, ,, novas, habilidades]




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\notxy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\notxy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\notxy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Lemmatização

In [97]:
# lemmatização utilizando spaCy, pegando o lemma_ dos tokens da ultima etapa.
# utilizando lemmatização, pois ela os preserva melhor 
# a semântica em comparação com stemming. 
# stemming não funciona tão bem quando é necessário manter a semântica das palavras.

def lemmatize_text(tokens):
    return [token.lemma_ for token in tokens]




lemmas = " ".join(lemmatize_text(tokens_no_stop))

print("Lemmas (primeiros 30):\n\n", lemmas[:300])


Lemmas (primeiros 30):

 aprendizagem maquina aprendizado maquina am segmento inteligencia artificial possuir elemento essencial comportamento inteligente , capacidade aprendizado . areo Am responsavel pesquisar metodo computacional adequar aquisicao novo conhecimento , novo habilidade novo forma organizacao conhecimento ja


# Chunking


### Recursívo

In [98]:
# técnica utilizada: chunking recursivo
# divide o texto em chunks utilizando RecursiveCharacterTextSplitter do LangChain

# corta o texto, tentando preservar a integridade semântica dos pedaços, 
# utilizando uma lista de separadores

# os separadores usados nesse caso foram: 
# dupla quebra de linha (para titulos e paragrafos), 
# quebra de linha padrão, 
# ponto e virgula para frases, 
# e espaços para palavras.

# isso visa ir do mais amplo para o mais específico.
# o chunk size de 100 foi escolhido pelo tamanho do texto, que é relativamente pequeno
# o overlap de 10 é 10% do chunk-size.           (10-20% do chunk-size é recomendado.)
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)




chunks_recursive = recursive_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_recursive)}\n\n", chunks_recursive)

Total de chunks: 24

 ['aprendizagem maquina aprendizado maquina am segmento inteligencia artificial possuir elemento', 'elemento essencial comportamento inteligente', ', capacidade aprendizado', '. areo Am responsavel pesquisar metodo computacional adequar aquisicao novo conhecimento', ', novo habilidade novo forma organizacao conhecimento ja existente', '. aprendizado possibilito sistema facar mesmo tarefa tarefa sobre mesmo populacao maneira eficiente', 'eficiente cada execucao', '. campo aprendizado maquina conceber questao construir programa', ', automaticamente melhor experiencia michell , 1997', '. segundo batista aprendizar maquina am subarea pesquisa importante inteligencia artificial ir pois', 'ir pois capacidade aprender essencial comportamento inteligente', '. Am tratar estudo metor computacional adquirir novo conhecimento', ', novo habilidade novo meio organizar conhecimento ja existente', '. 2003 , 11', '. aprendizado supervisionar ocorrer conjunto exemplo sao fornecer si

# Embedding

## Hugging Face SentenceTransformers

In [99]:
# transforma os chunks em embeddings, para que possam ser inseridos no banco vetorial.
# utiliza o modelo pré-treinado do sentence transformers para gerar embeddings

# escolhi o modelo "paraphrase-multilingual-MiniLM-L12-v2" por ser leve 
# e ter boa performance em múltiplos idiomas.
# nos testes que fiz, foi o que se saiu melhor.

model_name = "paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

def embed_hf(chunks):
    vectors = model.encode(chunks, show_progress_bar=True)

    print(f"\nDimensão: {vectors.shape}\n\nResultado: \n", vectors[:5])

    return vectors




embedding = embed_hf(chunks_recursive)


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.93it/s]


Dimensão: (24, 384)

Resultado: 
 [[-0.05333175 -0.17737345 -0.0476945  ...  0.15515457 -0.0615542
  -0.173321  ]
 [-0.00532783  0.15110852 -0.2187479  ...  0.46550682 -0.05833727
   0.04138125]
 [ 0.25355896 -0.11605787 -0.29423347 ...  0.17351723 -0.1914304
  -0.00337322]
 [-0.20642872 -0.02353211 -0.172113   ...  0.10338047 -0.11056297
  -0.100933  ]
 [-0.10843997 -0.18992469 -0.2598444  ...  0.05734963 -0.02787761
   0.18699992]]


# Banco Vetorial

## Criação/Inserção ChromaDb

In [100]:
# inicializa o cliente ChromaDB e cria uma coleção para armazenar os embeddings
client = chromadb.Client()
collection_name = "exemplo_colecao"

# resetando coleção caso exista (para testes)
if collection_name in [col.name for col in client.list_collections()]:
    client.delete_collection(collection_name)

collection = client.get_or_create_collection(
    name=collection_name,
    embedding_function=None,
    metadata={"hnsw:space": "cosine"}
)

# metadata especifica que vai utilizar o método de cosine, que é o método que
# o sentence-transformers e o modelo utilizam.



print(f"{collection.count()} documentos na coleção")

0 documentos na coleção


In [101]:
# Insere os embeddings na coleção do ChromaDB, criando IDs únicos para cada chunk
ids = [f"chunk_{i}" for i in range(len(chunks_recursive))]

collection.add(
    ids=ids,
    documents=chunks_recursive,
    embeddings=embedding
)




print("Embeddings inseridos com sucesso!")


Embeddings inseridos com sucesso!


## Busca Semântica

In [102]:
# realiza uma consulta de similaridade no banco vetorial utilizando uma pergunta
questions = ["o que é aprendizado de máquina?", "o que é aprendizado supervisionado?"]
for question in questions:
    # pega uma pergunta, e converte em embedding, no mesmo modelo usado para criar 
    # os embeddings dos chunks.
    question_embedding = model.encode([question])

    # realiza a consulta no banco vetorial utilizando o embedding da pergunta, 
    # e pega os 3 resultados mais similares.
    resultados = collection.query(
        query_embeddings=question_embedding.tolist(),
        n_results=3
    )

    print(f"\nPergunta: {question}")

    # os scores mostram a similaridade entre a pergunta e o chunk retornado. 
    # Scores mais baixos indicam maior similaridade. (0 é idêntico, 1 é muito diferente)
    for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
        print(f"(Score: {score}) - Documento: {doc}")


# as primeiras respostas (mais precisas) receberam score de 0.249... e 0.188...
# elas pegaram corretamente os trechos do texto com que eu esperava, e baseei a pergunta,
# então considero que a busca foi um sucesso.



Pergunta: o que é aprendizado de máquina?
(Score: 0.2497009038925171) - Documento: aprendizagem maquina aprendizado maquina am segmento inteligencia artificial possuir elemento
(Score: 0.27182525396347046) - Documento: . campo aprendizado maquina conceber questao construir programa
(Score: 0.43537425994873047) - Documento: , capacidade aprendizado

Pergunta: o que é aprendizado supervisionado?
(Score: 0.1887296438217163) - Documento: . aprendizado supervisionar ocorrer conjunto exemplo sao fornecer sistema respectivo classe
(Score: 0.2815517783164978) - Documento: , capacidade aprendizado
(Score: 0.30614417791366577) - Documento: . aprendizar supervisionar atividade predicao classificacao previsao classe discreta pre definido
